### Data setup

Connect to Google Drive

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 22.5MB/s 


In [0]:
from time import time
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, CuDNNGRU, Dropout, Bidirectional, Embedding, Flatten, Dense, GlobalMaxPooling1D, Conv1D
from keras.initializers import RandomUniform
from time import time
import keras
import math
import pandas as pd
from sklearn import model_selection
import nltk 
nltk.download('punkt')
from keras.preprocessing.sequence import pad_sequences
import sklearn
import tensorflow as tf
from keras.initializers import RandomUniform
import math
from keras.layers.embeddings import Embedding
from keras import backend as K
import numpy as np
from keras.regularizers import l1, l2, l1_l2

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Loads in data

In [0]:
glove_size = 300

glove_files = {
    50: {"id": '1WZ74wJa8_Ca5_PwxYalO6_kMrqBwo93i', 'name': 'glove.6B.50d.txt'},
    100: {"id": '1z540NC9cHtUWDppZUGWQloMTQQbMqJOX', 'name': 'glove.6B.100d.txt'},
    200: {"id": '1O9R5OsNMAU8IJL7bNprmI0HGzfvwngCQ', 'name': 'glove.6B.200d.txt'},
    300: {"id": '1rJ9GlJI4QRp6bmGTIk0hMKI-CQNU3osm', 'name': 'glove.6B.300d.txt'}
}

# downloaded = drive.CreateFile({'id': '1TUfn4whjpplGJMPwva7ePTFX0ELPvcx6'}) 
# downloaded.GetContentFile('val_30_70_features.csv') 
# validationData = pd.read_csv('val_30_70_features.csv', escapechar='\\', engine='python')

dataset_name = 'training'

data_files = {
    'validation': {"id": '1B9GWwTl3UyiMPr6B7klHz2TZxswz60xX', 'name': 'val_30_70.csv'},
    'training': {"id": '1eIwy9DL__7w15kuxId2kYE4j4Xi-YPs9', 'name': 'training_30_70.csv'},
}

downloaded = drive.CreateFile(data_files[dataset_name]) 
downloaded.GetContentFile(data_files[dataset_name]['name']) 
validationText = pd.read_csv(data_files[dataset_name]['name'], header=0)

glovedownload = drive.CreateFile(glove_files[glove_size])
glovedownload.GetContentFile(glove_files[glove_size]["name"])


In [0]:
# 90% training, 9% test, 1% validation
x_train, x_test, y_train, y_test = model_selection.train_test_split(validationText.loc[:,"text"], validationText.loc[:,"funny"], test_size=0.1, train_size=0.9, shuffle=True)


In [0]:

class WordCounter:
    def __init__(self):
        self.data = dict()
        self.global_id = 0

    def add(self, key):
        try:
            self.data[key][0] += 1
        except KeyError:
            self.data[key] = list()
            self.data[key].append(1)
            self.data[key].append(self.global_id)
            self.global_id += 1

    def get_count(self, key):
        try:
            val = self.data[key][0]
        except KeyError:
            val = None
        return val
      
    def get_id(self, key):
        try:
            val = self.data[key][1]
        except KeyError:
            val = None
        return val

    def pop(self, key):
        return self.data.pop(key)
      
    def __len__(self):
      return len(self.data.keys())
    
    def reset_global_ids(self):
      self.global_id = 0
      for key in self.data.keys():
        self.data[key][1] = self.global_id
        self.global_id += 1


In [0]:
def process_in_place(data, word_count=None, trim_tail=True, vect_len=100, lower_case=True):
    if word_count is None:
        word_count = WordCounter()
        for row in data:
          words = nltk.word_tokenize(row)
          if len(words) > vect_len:
            words = words[:vect_len]
          for w in words:
              if lower_case:
                w = w.lower()
              word_count.add(w)
            
    processed = list()
    for row in data:
      words = nltk.word_tokenize(row)
      if len(words) > vect_len:
          words = words[:vect_len]
      wc = 0
      for w in words:
          if lower_case:
            w = w.lower()
          count = word_count.get_count(w)
          if count is None:
              words[wc] = "<unk>"
          elif trim_tail and count == 1:
              words[wc] = "<unk>"
              word_count.pop(w)
              word_count.add("<unk>")
          wc += 1
      processed.append(words)
    word_count.reset_global_ids()
    return processed, word_count

In [0]:

def process_data(data, wc=None, trim_tail=True, vect_len=100):
    message = ""
    data_proc, data_wc = process_in_place(data, wc, trim_tail=trim_tail)
    print("Size of vocabulary: %d" % len(data_wc))
    print("Amount of unkown words: " + str(data_wc.get_count("<unk>")))
    sequences = list()
    for line in data_proc:
        sequence = list()
        for w in line:
          id = data_wc.get_id(w)
          if id is None:
             id = data_wc.get_id("<unk>")
          sequence.append(id)
        sequences.append(sequence)
#     print(sequence)
    X = pad_sequences(sequences, maxlen=vect_len, padding='post')
    return X, data_wc, data_proc


In [0]:
def measure(model, test_data, test_labels):
    metrics = model.evaluate(test_data, test_labels)
    for i in range(len(metrics)):
        print(str(model.metrics_names[i] + " = %f" % metrics[i]))
    print(str("perplexity = " + str(math.exp(metrics[0]))))
    
    predictions = model.predict_classes(test_data)
    f1 = sklearn.metrics.f1_score(test_labels, predictions)
    print('F1:', f1)
    ba = sklearn.metrics.balanced_accuracy_score(test_labels, predictions)
    print("Bal Acc:", ba)

In [0]:
# Improved upon from https://josephguhlin.com/importing-glove-embeddings-into-tensorflow/
def dawn_glove(wc, embedding_dim = 50):
    with open("glove.6B."+str(embedding_dim)+"d.txt", 'r') as f:
      vectors = {}
      for line in f:
          vals = line.rstrip().split(' ')
          if wc.get_id(vals[0]) is not None:
              vectors[vals[0]] = [float(x) for x in vals[1:]]

    vocab_size = len(vectors.keys())

    vocab = {w: idx for idx, w in enumerate(vectors.keys())}
    vector_dim = len(vectors["the"])
    W = np.zeros((vocab_size, vector_dim))
    for word, v in vectors.items():
      if word == '':
          continue
      W[vocab[word], :] = v

    # normalize each word vector to unit variance
    W_norm = np.zeros(W.shape)
    d = (np.sum(W ** 2, 1) ** (0.5))

    W_norm = (W.T / d).T
    return W_norm

In [0]:
# Used from https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
  
def specificity(y_true, y_pred):
    """TN/N"""
    tn = tf.metrics.true_negatives(y_true, y_pred)
    fp = tf.metrics.false_positives(y_true, y_pred)
    n = tf.math.add(tn, fp)
    result = tf.math.divide(tn, n)
    return result

def f1(y_true, y_pred):
#     return sklearn.metrics.f1_score(y_true, y_pred)
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

def bal_acc(y_true, y_pred):
    r = recall(y_true, y_pred)
    s = specificity(y_true, y_pred)
#     s = K.metrics.specificity_at_sensitivity(y_true, y_pred, 0)
    return K.math.divide(tf.math.add(r, s), 2)
#     return sklearn.metrics.balanced_accuracy_score(list(y_true), list(y_pred))
    

In [0]:
sent_len = 150
doc_amount = len(x_train)
# doc_amount = 100000
# Isolated this because it takes a long time
X, data_wc, data_proc = process_data(x_train[:doc_amount].values, wc=None, trim_tail=True, vect_len=sent_len)
test_X, test_data_wc, test_data_proc = process_data(x_test[:doc_amount].values, wc=data_wc, trim_tail=True, vect_len=sent_len)


Size of vocabulary: 122311
Amount of unkown words: 257900
Size of vocabulary: 122311
Amount of unkown words: 257900


### FFNN

In [0]:
def learn_FFNN(train_data, train_labels, epochs=10, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True):
  start_time = time()
#   print(init.shape)
  init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
  model = Sequential()
  model.add(Embedding(len(data_wc), 300, input_length=vect_len, embeddings_initializer=init))
#   model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
  model.add(Flatten())
  model.add(Dense(500, activation=tf.nn.relu))
  model.add(Dropout(0.3))
  model.add(Dense(100, activation=tf.nn.relu))
  model.add(Dropout(0.3))
  model.add(Dense(1, activation=tf.sigmoid))
  print(str(model.summary()))
  # compile network
  learningRate = 0.00001
  decay = learningRate/epochs
  adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
  model.compile(loss=loss, optimizer=adamOpt, metrics=[f1])
  # fit network
  model.fit(train_data, train_labels, epochs=epochs, validation_split=0.1, verbose=2)
  print(str("runtime: " + str(time() - start_time)))
  return model

In [0]:
# init = dawn_glove(data_wc, glove_size)
init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
model2 = learn_FFNN(X, y_train[:doc_amount].values, epochs=20, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
measure(model2, test_X, y_test[:doc_amount].values)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 300)          36789300  
_________________________________________________________________
flatten_1 (Flatten)          (None, 45000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               22500500  
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               50100     
_________________________________________________________________
dropout_2 (Dropout)  

In [0]:
# init = dawn_glove(data_wc, glove_size)
# model10 = learn_FFNN(X, y_train[:doc_amount].values, epochs=25, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(model10, test_X, y_test[:doc_amount].values)

### LSTM

In [0]:
# adapted from https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
def learn_LSTM(train_data, train_labels, epochs=10, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True):
  start_time = time()
  print(init.shape)
  model = Sequential()
  model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
  model.add(CuDNNLSTM(model_size, return_sequences=True))
  model.add(Dropout(0.25))
  model.add(CuDNNLSTM(model_size))
  model.add(Dropout(0.25))
  model.add(Dense(vect_len, activation=tf.sigmoid))
  model.add(Dense(1, activation=tf.sigmoid))
  print(str(model.summary()))
  # compile network
  model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
  # fit network
  model.fit(train_data, train_labels, epochs=epochs)
  print(str("runtime: " + str(time() - start_time)))
  return model



In [0]:
# init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
init = dawn_glove(data_wc, glove_size)
model2 = learn_LSTM(X, y_train[:doc_amount].values, epochs=20, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
measure(model3, test_X, y_test[:doc_amount].values)
model3 = learn_LSTM(X, y_train[:doc_amount].values, epochs=20, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=True)
measure(model4, test_X, y_test[:doc_amount].values)

### BiLSTM

In [0]:
def learn_BiLSTM(train_data, train_labels, epochs=10, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True):
  start_time = time()
  init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
  model = Sequential()
  model.add(Embedding(len(data_wc), 300, input_length=vect_len, embeddings_initializer=init))
#   model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
  model.add(Bidirectional(CuDNNLSTM(model_size, return_sequences=True), input_shape=(vect_len, 1)))
  model.add(Dropout(0.2))
  model.add(Bidirectional(CuDNNLSTM(model_size, return_sequences=True), input_shape=(vect_len, 1)))
  model.add(Flatten())
  model.add(Dropout(0.2))
  model.add(Dense(100, activation=tf.nn.relu))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation=tf.sigmoid))
  print(str(model.summary()))
  # compile network
  learningRate = 0.00001
  decay = learningRate/epochs
  adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
  model.compile(loss=loss, optimizer='adam', metrics=[f1])
  # fit network
  model.fit(train_data, train_labels, epochs=epochs, validation_split=0.05, verbose=2)
  print(str("runtime: " + str(time() - start_time)))
  return model


In [0]:
# # init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
# init = dawn_glove(data_wc, glove_size)
# model4 = learn_BiLSTM(X, y_train[:doc_amount].values, epochs=40, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(model4, test_X, y_test[:doc_amount].values)

In [0]:
# # init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
# init = dawn_glove(data_wc, glove_size)
# model11 = learn_BiLSTM(X, y_train[:doc_amount].values, epochs=25, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(model11  , test_X, y_test[:doc_amount].values)

In [0]:
init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
# init = dawn_glove(data_wc, glove_size)
model30 = learn_BiLSTM(X, y_train[:doc_amount].values, epochs=1, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
measure(model30, test_X, y_test[:doc_amount].values)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 300)          36789300  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 150, 200)          321600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 150, 200)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 150, 200)          241600    
_________________________________________________________________
flatten_2 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 30000)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               3000100   
__________

### CNN

In [0]:
# Ideas from https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74
def learn_CNN(train_data, train_labels, epochs=10, embed_size=100, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True, kgram=2):
    init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
    model = Sequential()
    model.add(Embedding(len(data_wc), 300, input_length=vect_len, embeddings_initializer=init))
#     model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
    model.add(Conv1D(filters=100, kernel_size=kgram, padding='valid', activation='relu', strides=1))
    model.add(Dropout(0.2))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(50 ,activation=tf.nn.relu))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    print(str(model.summary()))
    learningRate = 0.001
    decay = learningRate/epochs
    adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
    model.compile(loss=loss, optimizer=adamOpt, metrics=[f1])
    model.fit(train_data, train_labels, epochs=epochs, validation_split=0.1, verbose=2)
    return model

In [0]:
# init = dawn_glove(data_wc, glove_size)
# model23 = learn_CNN(X, y_train[:doc_amount].values, epochs=15, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False, kgram=2)
# measure(model23, test_X, y_test.values)

In [0]:
# init = dawn_glove(data_wc, glove_size)
init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
model5 = learn_CNN(X, y_train[:doc_amount].values, epochs=15, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False, kgram=3)
measure(model5, test_X, y_test.values)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 300)          36766800  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 148, 100)          90100     
_________________________________________________________________
dropout_3 (Dropout)          (None, 148, 100)          0         
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total para

In [0]:
# init = dawn_glove(data_wc, glove_size)
# model22 = learn_CNN(X, y_train[:doc_amount].values, epochs=10, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False, kgram=4)
# measure(model22, test_X, y_test.values)

In [0]:
# init = dawn_glove(data_wc, glove_size)
# model22 = learn_CNN(X, y_train[:doc_amount].values, epochs=10, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False, kgram=10)
# measure(model22, test_X, y_test.values)

###BiGRU

In [0]:
def learn_BiGRU(train_data, train_labels, epochs=10, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True):
  start_time = time()
  # define model
#   print(init.shape)
# init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
  model = Sequential()
#   model.add(Embedding(len(data_wc), 300, input_length=vect_len, embeddings_initializer=init))
  model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
  model.add(Bidirectional(CuDNNGRU(model_size, return_sequences=True), input_shape=(vect_len, 1)))
  model.add(Dropout(0.2))
  model.add(Bidirectional(CuDNNGRU(model_size), input_shape=(vect_len, 1)))
  model.add(Dropout(0.2))
  model.add(Dense(100, activation=tf.nn.relu))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation=tf.sigmoid))
  print(str(model.summary()))
  # compile network
  learningRate = 0.001
  decay = learningRate/epochs
  adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
  model.compile(loss=loss, optimizer=adamOpt, metrics=[f1])
  # fit network
  model.fit(train_data, train_labels, epochs=epochs, validation_split=0.1, verbose=2)
  print(str("runtime: " + str(time() - start_time)))
  return model

In [0]:
init = dawn_glove(data_wc, glove_size)
model20 = learn_BiGRU(X, y_train[:doc_amount].values, epochs=5, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
predictions = model20.predict_classes(test_X)
measure(model20, test_X, y_test[:doc_amount].values)
confusionMatrix = sklearn.metrics.confusion_matrix(predictions, y_test[:doc_amount].values)
print('Confusion Matrix:', confusionMatrix)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 300)          19426500  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 150, 200)          241200    
_________________________________________________________________
dropout_4 (Dropout)          (None, 150, 200)          0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200)               181200    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
__________

In [0]:
# init = dawn_glove(data_wc, glove_size)
# model21 = learn_BiGRU(X, y_train[:doc_amount].values, epochs=10, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(model21, test_X, y_test[:doc_amount].values)

In [0]:
# init = dawn_glove(data_wc, glove_size)
# init = RandomUniform(minval=-0.05, maxval=0.05, seed=None)
# model40 = learn_BiGRU(X, y_train[:doc_amount].values, epochs=5, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(model40, test_X, y_test[:doc_amount].values)

In [0]:
print(model40)


predictions = model40.predict_classes(test_X[10:20])
labels = y_test.values[10:20]
text_data = x_test[10:20]

print(predictions)
print(labels)

print(predictions[2:3])
print(labels[2:3])
print(text_data[2:3].values)

# print('False negative:', x_test[18:19])

# print(text_data)

# print('True negative:', x_test.iloc[0])
# print('True negative:', x_test.iloc[1])
# print('True positive:', x_test.iloc[2])
# print('False positive:', x_test.iloc[3])
# print('True positive:', x_test.iloc[6])




### Ensemble - BiGRU, BiLSTM, CNN

In [0]:
# def learn_BiLSTM(train_data, train_labels, epochs=10, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True):
#   start_time = time()
#   print(init.shape)
#   model = Sequential()
#   model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
#   model.add(Bidirectional(CuDNNLSTM(model_size, return_sequences=True), input_shape=(vect_len, 1)))
#   model.add(Dropout(0.2))
#   model.add(Bidirectional(CuDNNLSTM(model_size, return_sequences=True), input_shape=(vect_len, 1)))
#   model.add(Flatten())
#   model.add(Dropout(0.2))
#   model.add(Dense(100, activation=tf.nn.relu))
#   model.add(Dropout(0.2))
#   model.add(Dense(1, activation=tf.sigmoid))
#   print(str(model.summary()))
#   # compile network
#   learningRate = 0.00001
#   decay = learningRate/epochs
#   adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
#   model.compile(loss=loss, optimizer='adam', metrics=[f1])
#   # fit network
#   model.fit(train_data, train_labels, epochs=epochs, validation_split=0.1, verbose=2)
#   print(str("runtime: " + str(time() - start_time)))
#   return model
# # Ideas from https://towardsdatascience.com/another-twitter-sentiment-analysis-with-python-part-11-cnn-word2vec-41f5e28eda74
# def learn_CNN(train_data, train_labels, epochs=10, embed_size=100, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True, kgram=2):
#     model = Sequential()
#     model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
#     model.add(Conv1D(filters=100, kernel_size=kgram, padding='valid', activation='relu', strides=1))
#     model.add(Dropout(0.2))
#     model.add(GlobalMaxPooling1D())
#     model.add(Dense(50 ,activation=tf.nn.relu))
#     model.add(Dropout(0.2))
#     model.add(Dense(1, activation='sigmoid'))
#     learningRate = 0.001
#     decay = learningRate/epochs
#     adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
#     model.compile(loss=loss, optimizer=adamOpt, metrics=[f1])
#     model.fit(train_data, train_labels, epochs=epochs, validation_split=0.1, verbose=2)
#     return model
# def learn_BiGRU(train_data, train_labels, epochs=10, model_size=100, vect_len=100, loss='binary_crossentropy', weights=None, train_embedding=True):
#   start_time = time()
#   # define model
#   print(init.shape)
#   model = Sequential()
#   model.add(Embedding(init.shape[0], init.shape[1], input_length=vect_len, weights=[weights], trainable=train_embedding))
#   model.add(Bidirectional(CuDNNGRU(model_size, return_sequences=True), input_shape=(vect_len, 1)))
#   model.add(Dropout(0.2))
#   model.add(Bidirectional(CuDNNGRU(model_size), input_shape=(vect_len, 1)))
#   model.add(Dropout(0.2))
#   model.add(Dense(100, activation=tf.nn.relu))
#   model.add(Dropout(0.2))
#   model.add(Dense(1, activation=tf.sigmoid))
#   print(str(model.summary()))
#   # compile network
#   learningRate = 0.001
#   decay = learningRate/epochs
#   adamOpt = keras.optimizers.Adam(lr=learningRate, decay=decay)
#   model.compile(loss=loss, optimizer=adamOpt, metrics=[f1])
#   # fit network
#   model.fit(train_data, train_labels, epochs=epochs, validation_split=0.1, verbose=2)
#   print(str("runtime: " + str(time() - start_time)))
#   return model

In [0]:
# init = dawn_glove(data_wc, glove_size)

# BiLSTMModel = learn_BiLSTM(X, y_train[:doc_amount].values, epochs=4, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(BiLSTMModel, test_X, y_test.values)
# BiGRUModel = learn_BiGRU(X, y_train[:doc_amount].values, epochs=10, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False)
# measure(BiGRUModel, test_X, y_test.values)
# CNNModel = learn_CNN(X, y_train[:doc_amount].values, epochs=1, model_size=100, vect_len=sent_len, loss='binary_crossentropy', weights=init, train_embedding=False, kgram=3)
# measure(CNNModel, test_X, y_test.values)


# combo = np.append(BiLSTMModel.predict_classes(test_X), BiGRUModel.predict_classes(test_X), axis=1)
# combo = np.append(combo, CNNModel.predict_classes(test_X), axis=1)
# modePredictions = pd.DataFrame(combo).mode(axis='columns')
# ensembleF1 = sklearn.metrics.f1_score(modePredictions.values, y_test[:doc_amount].values)
# print('Ensemble F1 Score:', str(ensembleF1))